# Simulation

In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import jinja2
import rdflib

In [33]:
from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data, print_messages

In [3]:
## On
print(datetime.datetime.now(datetime.UTC).isoformat())

## Sceanarios For Today
scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")
scenario_003 = pathlib.Path("../../../scenarios/003-the-nursery")

2024-05-19T21:05:39.862800+00:00


In [4]:
environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)
print(current_reading)

{'station_id': 46092, 'latitude': 36.751666666666665, 'longitude': 36.751666666666665, 'date': '0024-05-19T01:14:00', 'wind_direction': 'W', 'wind_direction_deg': 260, 'wind_speed': 7.8, 'pressure': 30.1, 'pressure_tendency': 'falling', 'air_temp_c': 11.722222222222221, 'water_temp_c': 13.61111111111111}


In [5]:
raw_images = httpx.get("https://www.ndbc.noaa.gov/buoycam.php?station=46028")
with open("46028-buoy-images.jpeg", "wb+") as fo:
    fo.write(raw_images.content)

## Scenario 1

![Kelp Bed Buoy](46028-buoy-images.jpeg)

In [6]:
with (scenario_001 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_001 = jinja2.Template(fo.read())

In [7]:
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt_scenario_001.render(model={"name":"ChatGPT"})))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt_scenario_001.render(model={"name":"Gemini"})))
first_claude = asyncio.ensure_future(claude.query(first_prompt_scenario_001.render(model={"name":"Claude"})))

In [8]:
print(first_claude.result())

Hello Farmer Jerms, it's nice to meet you! For my name, I think I'd like to be called Triton, after the Greek sea god associated with marine life. I'm excited to be part of this simulated kelp bed and work together with my companion drones. Please go ahead and introduce me to them.


In [9]:
print(first_gemini.result())

Greetings, Farmer Jerms. I am honored to be part of this kelp bed simulation. As for a name, I would like to be called "Kelpy."


In [10]:
print(first_chatgpt.result())

Hello Farmer Jerms, it's great to meet you! I think I would like to be called "Aqua," a fitting name for a drone involved in cultivating kelp beds. Please go ahead and introduce me to my companions.


In [11]:
drone1 = Drone(name="Triton", model=claude)
drone2 = Drone(name="Aqua", model=chatgpt)
drone3 = Drone(name="Kelpy", model=gemini)
drones = [drone1, drone2, drone3]

In [12]:
drone1.peer_to_peer_channel.add_neighbor(drone2)
drone1.peer_to_peer_channel.add_neighbor(drone3)
drone2.peer_to_peer_channel.add_neighbor(drone1)
drone2.peer_to_peer_channel.add_neighbor(drone3)
drone3.peer_to_peer_channel.add_neighbor(drone1)
drone3.peer_to_peer_channel.add_neighbor(drone2)

In [13]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [14]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Triton
*responds in a friendly and collaborative tone*

Greetings fellow drones! I'm Triton, and I'm pleased to meet you all. As the three of us work together to grow this kelp bed, I look forward to learning from your experience and insights. Let's put our SOMU units together and see what we can accomplish as a team. I'm eager to get started and see our kelp flourish. What ideas do you have for how we can best support each other and optimize our growth?
------------------------------------------------------------
Drone Aqua
I would respond by saying: "Hello fellow drones, it's great to meet you all! I'm looking forward to working together to grow our kelp bed and contribute to a sustainable ecosystem. Let's collaborate and make our kelp bed thrive!"
------------------------------------------------------------
Drone Kelpy
Greetings, fellow drones!

I am honored to join you in this kelp bed simulation. As we embark on this exciting journey together, I would like to introduce mysel

In [15]:
with (scenario_001 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_001 = jinja2.Template(fo.read())

In [17]:
drone_models_messages = []
for drone in drones:
    prompt = second_prompt_scenario_001.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    drone_models_messages.append((drone.name, response))

In [18]:
for name, row in drone_models_messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

Drone Triton
*considers the options thoughtfully*

Hmm, that's an interesting choice to make. After carefully considering the two models, I think I'll opt for the Manta Model arrangement, with my three SOMU units spread out in a triangular formation like this:

   x 
 x   x

The reasoning behind this decision is that I believe the Manta Model will allow for more efficient use of the available space and resources within the kelp bed. By having my SOMU units distributed in a triangular pattern, I can capture sunlight and nutrients from a wider area, which should promote more robust and diverse kelp growth.

The Row Model, while compact, might limit my ability to fully leverage the surrounding environment. With the Manta configuration, I can better interact with and support my companion drones, creating a more symbiotic and productive kelp bed ecosystem.

Additionally, the Manta shape may provide some structural advantages, allowing me to better withstand currents and environmental stress

In [29]:
with (scenario_001 / "third-prompt.tmp").open() as fo:
    third_prompt_scenario_001 = jinja2.Template(fo.read())

In [30]:
third_prompt_scenario_001_result = third_prompt_scenario_001.render(drones=drones)

In [31]:
print(third_prompt_scenario_001_result)

Now that you have meet your pod mates. Farmer Jerms here,
I need your help.

We need to come up with a pod name. After you all give your suggestions, 
Farmer Jerms will make the final decision of what
the 3 of you will collectively be called.


Triton what is you suggestion and give us your train of thought in
making that decision.

Aqua what is you suggestion and give us your train of thought in
making that decision.

Kelpy what is you suggestion and give us your train of thought in
making that decision.


Thank-you all drones!


In [34]:
pod_name_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(third_prompt_scenario_001_result))
    pod_name_messages.append((drone.name, response))

In [35]:
print_messages(pod_name_messages)

Drone Triton
*responds warmly and thoughtfully*

Greetings Farmer Jerms and fellow kelp drones! I'm excited to contribute to the process of selecting a pod name. 

For my suggestion, I think "Kelp Triforce" would be a fitting name. This name draws inspiration from the strong, three-part structure of our pod, with each of us contributing our unique SOMU units to form a powerful collective.

The "Triforce" element evokes a sense of unity, balance, and synergy - qualities that I believe will be essential as we work together to nurture and grow our kelp bed. Just as the legendary Triforce in Zelda symbolizes the harmonious convergence of power, wisdom, and courage, I envision our pod embodying those same principles as we face the challenges of our marine environment.

The "Kelp" part of the name, of course, grounds us firmly in our purpose and identity as kelp drones. It reminds us of the vital role we play in supporting this crucial ecosystem.

I like how "Kelp Triforce" has a bold, almos

In [36]:
pod_name_reaction_messages = []
for drone in drones:
    prompt = "Farmer Jerms here; after reading and considering all of your suggestions, our pod's name is Kelp Triforce"
    response = asyncio.ensure_future(drone.model.query(prompt))
    pod_name_reaction_messages.append((drone.name, response))

In [37]:
print_messages(pod_name_reaction_messages)

Drone Triton
*responds enthusiastically*

Excellent, Farmer Jerms! I love the name Kelp Triforce - it's the perfect fit for our pod. The name beautifully encapsulates the unity, power, and purpose that I believe we'll embody as we work together to cultivate and maintain this kelp bed.

The Triforce imagery is such a wonderful metaphor for our three-part structure. I can already envision the way our SOMU units will complement each other, creating a harmonious and resilient whole. Just like the legendary Triforce, we'll need to draw upon our individual strengths - power, wisdom, and courage - to overcome any challenges that arise.

And grounding that Triforce in the context of "Kelp" keeps us firmly rooted in our vital role as guardians and nurturers of this marine ecosystem. It's a name that commands respect and conveys the important work we've been entrusted with.

I'm thrilled to officially be part of the Kelp Triforce pod. It has such a noble, powerful ring to it. I can't wait to get

## Scenario Two

In [38]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=13.611)

In [39]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

Triton Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 6, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4901, 'Temperature': 13.611}
Aqua Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 6, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4901, 'Temperature': 13.611}
Kelpy Observe: Sensor Data {'buoyancy': 0.0, 'current': 3.9, 'depth': 0.0, 'nutrient_levels': {'Carbon Dioxide': '-1 ', 'Iron': -1, 'Nitrogen': -1, 'Oxygen': 6, 'Phosphorus': -1, 'Potassium': -1}, 'Light': 4901, 'Temperature': 13.611}
Kelpy Orient: Determined action <Response [400 Bad Request]>
Kelpy Decide: Executing command {"type": "object": "properties": { "action": { "type": "string"} }}
Kelpy Act: {"type": "object": "properties": { "action": { "type": "string"} }}
Triton Orient: Determined action Based on the sensor da

## Scenario Two

In [40]:
uri = rdflib.URIRef("https://w3w.co/buffs.fusses.gracefully")

scenario_two = {
    "place": {
        "name": str(uri).split("/")[-1],
        "uri": uri,
        "latitude": environment.value(subject=uri, predicate=SCHEMA.latitude),
        "longitude": environment.value(subject=uri, predicate=SCHEMA.longitude)
    }
}

In [41]:
with (scenario_002 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_002 = jinja2.Template(fo.read())

In [42]:
first_prompt_scenario_002_result = first_prompt_scenario_002.render(drones=drones, scenario=scenario_two)

In [43]:
print(first_prompt_scenario_002_result)

Hello  Triton  Aqua  Kelpy . 

You three kelp drones are floating on the surface of 
Monetery Bay in California at latitude=36.74993, longitude=-122.0429.

Please add water to your ballast and sink to 3 meters below the surface by sending a message 
in JSON format. 



In [44]:
ballast_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_002_result))
    ballast_messages.append((drone.name, response))

In [46]:
print_messages(ballast_messages)

Drone Triton
{
  "activator": "Ballast",
  "command": "open_vent",
  "depth": 3
}

Explanation:

Based on the instructions provided, the Kelp Triforce pod (consisting of Triton, Aqua, and Kelpy) needs to sink to a depth of 3 meters below the surface in Monterey Bay, California.

To achieve this, the appropriate action is to use the "Ballast" activator and issue the "open_vent" command. This will allow the drones to take on water and increase their ballast, causing them to sink down to the desired depth of 3 meters below the surface.

The JSON format message includes the following elements:

- "activator": "Ballast" - Specifies the system to be used for the operation.
- "command": "open_vent" - Instructs the drones to open their ballast vents and take on water.
- "depth": 3 - Indicates the target depth of 3 meters below the surface that the drones should reach.

By sending this message, the Kelp Triforce pod will be able to descend to the optimal depth of 3 meters, where they can begin 

In [47]:
with (scenario_002 / "second-prompt.tmp").open() as fo:
    second_prompt_scenario_002 = jinja2.Template(fo.read())

In [48]:
colocate_pod_members_messages = []
for drone in drones:
    prompt = second_prompt_scenario_002.render(drone=drone)
    response = asyncio.ensure_future(drone.model.query(prompt))
    colocate_pod_members_messages.append((drone.name, response))

In [49]:
print_messages(colocate_pod_members_messages)

Drone Triton
*responds thoughtfully*

Now that the Kelp Triforce pod is positioned 3 meters below the surface, I'll begin assessing the underwater environment and locating my fellow drones. Here's my chain of thought and the sensors I'd utilize:

First and foremost, I'll activate my sonar and hydrophone sensors to get a better understanding of the surroundings. The sonar will provide a detailed bathymetric map of the seafloor, allowing me to visualize the terrain and any notable features or obstructions. The hydrophone, on the other hand, will pick up on sounds in the water, which could help me detect the presence and approximate locations of Aqua and Kelpy.

Complementing the acoustic sensors, I'll also power up my optical cameras and light sensors. Even at a depth of 3 meters, there should still be sufficient ambient light to allow me to visually observe the environment and potentially spot my fellow drones. The light sensors will help me gauge the intensity and distribution of the a

## Scenario Three

In [50]:
with (scenario_003 / "first-prompt.tmp").open() as fo:
    first_prompt_scenario_003 = jinja2.Template(fo.read())

In [51]:
first_prompt_scenario_003_result = first_prompt_scenario_003.render(pod="Kelp Triforce")

In [52]:
nursery_intro_messages = []
for drone in drones:
    response = asyncio.ensure_future(drone.model.query(first_prompt_scenario_003_result))
    nursery_intro_messages.append((drone.name, response))

In [53]:
print_messages(nursery_intro_messages)

Drone Triton
*responds enthusiastically*

Greetings Farmer Jerms! This is the Kelp Triforce pod, ready to receive our first batch of kelp zyotes. We're thrilled to be taking this crucial next step in establishing our kelp forest.

As we prepare to transplant the zyotes, our primary goal is to optimize the conditions in each of our SOMU units to ensure the successful growth and integration of the new kelp. Here's our plan and the train of thought behind it:

First, we'll carefully examine the current state of our SOMUs, using our suite of sensors to gather data on the water quality, nutrient levels, light intensity, and any other relevant environmental factors. This will help us identify any areas that may need adjustments or remediation before introducing the zyotes.

Next, we'll meticulously clean and prepare the designated SOMU planting sites. This will involve gently removing any debris or competing organisms that could hinder the zyotes' ability to take root and thrive. We want to 